In [ ]:
import hana_ml.dataframe as dataframe
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
import pandas as pd
import warnings
from itertools import permutations
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
conn = NotebookConnectionContext(connectionId = 'hanapoc')

Get data

In [ ]:
sql = (
"select "
"top 2000 * "
"from VW_MES_CHANGEOVER_SUMMARY "
"where WORKCENTRE = 'RIGID05' "
#"and CHANGEOVER_COUNT > 2 "
#"and MATERIAL_FROM = 'RKC1230' "
#"and MATERIAL_TO = 'RKC1133'"
#"order by AVG_CHANGEOVER_DURATION desc"
#"and MATERIAL_FROM = 'RKC8134' "
"order by AVG_CHANGEOVER_DURATION desc"

)
    
po_sel = conn.sql(sql)
pos = po_sel.collect()
pos

Formatting

In [ ]:
un_from = pos.drop_duplicates(['MATERIAL_FROM'])['MATERIAL_FROM']
un_to = pos.drop_duplicates(['MATERIAL_TO'])['MATERIAL_TO']
comb = pd.DataFrame(un_from.append(un_to), columns=['MATERIAL_CONCAT']).drop_duplicates(['MATERIAL_CONCAT'])
comb
comb_chan = comb + '_' + comb
comb_chan
comb_chan['AVG_CHANGEOVER_DURATION'] = 0
comb_chan['CHANGEOVER_COUNT'] = 0
inter_pos = pos.drop(['PLANT', 'WORKCENTRE','OPERATION','MATERIAL_FROM','MATERIAL_TO'], axis=1)
inter = inter_pos.append(comb_chan, ignore_index=True)

Define function to perform permutations given a start, bundle and end

In [ ]:
def prod_plan(start, demand, end):
    p = permutations(demand)
    data = pd.DataFrame(columns=['sequence'])
    k=0
    for i in list(p):
        df = pd.DataFrame()
        i = (start,) + i + (end,)
        res = tuple(j + '_' + k for j, k in zip(i, i[1:]))
        list_res = list(res)
        
        df['changeover'] = pd.DataFrame(list_res)  
        listToStr = '_'.join([str(elem) for elem in i])
        df['sequence'] = '_'.join([str(elem) for elem in i])
        if k == 0:
            df['orig'] = 1
        else:
            df['orig'] = 0
        data = data.append(df, ignore_index=True)
        k = k+1
    data1 = data.drop_duplicates(subset=['sequence', 'changeover'],keep='first')
#    print(data1)
    return(data1)

Define a function to aggregate the changeover time for sequences

In [ ]:
def calc_times(dat):
    times = dat.merge(inter, left_on='changeover', right_on='MATERIAL_CONCAT', how='left')
    times['AVG_CHANGEOVER_DURATION'] = times['AVG_CHANGEOVER_DURATION'].astype(float)
    times['flag'] = np.where(times.notna().all(axis=1), 0, 1)
#    print(times)
    result = times.groupby(['sequence'])['AVG_CHANGEOVER_DURATION','flag','orig'].sum()
    result_final = result[result.flag == 0].sort_values(by=['AVG_CHANGEOVER_DURATION'])
    result_final.loc[result_final['orig'] > 0, 'orig'] = 'original'
#    print(result_final)
    result_final.drop('flag', axis=1, inplace=True)
    result_final['orig'].iloc[0] = 'best'
    result_final['orig'].iloc[-1] = 'worst'

    result_final.insert(0, 'id', range(1, 1 + len(result_final)))
    return(result_final)

Combine functions to simulate all (possible) sequences

In [ ]:
#opt, best, worst = calc_times(prod_plan('RKC8134',['RKC8707','RKC8133','HCB1133','RKC1230'] ,'RKC8705'))
#opt = calc_times(prod_plan('RKC8134',['RKC8707','RKC8133','HCB1133','HCB1133'] ,'RKC8705'))
#opt = calc_times(prod_plan('RKC1230',['HCB1134','HCB1133','RKC8707','HCB1230','RKC1230','RKC1705','RKC8133','RKC1230'],'RKC8230'))
opt = calc_times(prod_plan('RKC8230',['RKC8707','RKC8230','HCB1133','RKC1230'],'RKC1133'))
opt

Plot all simulated production sequences

In [ ]:
plt.xticks(opt['id'])
sns.scatterplot(x="id", y="AVG_CHANGEOVER_DURATION", data=opt, hue="orig")

Calculate the factorial(n!) of given number n

In [ ]:
n = 12
fact = 1
  
for i in range(1,n+1):
    fact = fact * i
      
print ("The factorial is : ",end="")
print (fact)